In [1]:
!pip install yfinance
!pip install fredapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 17.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
   

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import fredapi
import datetime as dt
from itertools import chain
from mpl_toolkits.mplot3d import Axes3D
import plotly
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"

In [3]:
# choose a ticker and get data via yfinance
ticker = "ATVI"
stock = yf.Ticker(ticker)
# store maturities
lMaturity = list(stock.options)

In [4]:
# get current date
today = dt.datetime.now().date()
# empty list for days to expiration
lDTE = []
# empty list to store data for calls
lData_calls = []
lData_puts = []
# loop over maturities
for maturity in lMaturity:
    # maturity date
    maturity_date = dt.datetime.strptime(maturity, '%Y-%m-%d').date()
    # DTE: difference between maturity date and today
    lDTE.append((maturity_date - today).days)
    # store call data
    lData_calls.append(stock.option_chain(maturity).calls)
    lData_puts.append(stock.option_chain(maturity).puts)

In [5]:
# create empty lists to contain unlisted data
lStrike = []
lDTE_extended = []
lImpVol = []
for i in range(0,len(lData_calls)):
    # append strikes to list
    lStrike.append(lData_puts[i]["strike"])
    # repeat DTE so the list has same length as the other lists
    lDTE_extended.append(np.repeat(lDTE[i], len(lData_puts[i])))
    # append implied volatilities to list
    lImpVol.append(lData_puts[i]["impliedVolatility"])
    
# unlist list of lists
lStrike = list(chain(*lStrike))
lDTE_extended = list(chain(*lDTE_extended))
lImpVol = list(chain(*lImpVol))

In [6]:
#pandas.pivot_table(data, values=None, index=None,
GG=pd.DataFrame(data=lImpVol)
GG['Strike'] = lStrike
GG['DTE'] = lDTE_extended
GG.columns=[ 'IV','Strike', 'DTE']
GG_pivot=GG.pivot_table(GG, index='Strike', columns='DTE')
GG_pivot=GG_pivot.droplevel(level=0, axis=1)

In [7]:
#Force Interpolate data
GG_pivot.interpolate(method='linear', inplace=True)

In [8]:
GG_pivot

DTE,1,8,15,22,29,36,50,85,113,141,204,358,694,722
Strike,,,,,,,,,,,,,,
30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.250007,NaN,NaN
32.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.545903,NaN,NaN
35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.518560,NaN,NaN
37.5,NaN,NaN,NaN,0.500005,NaN,NaN,1.050786,NaN,NaN,0.667972,NaN,0.125009,0.125009,0.125009
40.0,NaN,0.500005,NaN,0.500005,NaN,NaN,0.953125,NaN,0.684573,0.486984,NaN,0.458257,0.276741,0.353156
42.5,NaN,0.500005,NaN,0.500005,NaN,NaN,0.950684,NaN,0.616215,0.305997,NaN,0.125009,0.428473,0.358039
45.0,NaN,0.500005,NaN,0.500005,NaN,NaN,0.948243,NaN,0.547856,0.125009,NaN,0.125009,0.062509,0.395544
47.5,NaN,0.500005,NaN,0.500005,NaN,NaN,0.250007,NaN,0.541020,0.125009,NaN,0.375006,0.366583,0.433050
50.0,NaN,0.500005,NaN,0.250007,NaN,0.250007,0.250007,NaN,0.576420,0.125009,NaN,0.062509,0.328620,0.062509


In [9]:
fred = fredapi.Fred(api_key='5b460e8b16366cedadb8dc285dfedc26')

In [10]:
fred_data = fred.get_series('FEDFUNDS')

In [12]:
fred_data

1954-07-01    0.80
1954-08-01    1.22
1954-09-01    1.07
1954-10-01    0.85
1954-11-01    0.83
              ... 
2022-08-01    2.33
2022-09-01    2.56
2022-10-01    3.08
2022-11-01    3.78
2022-12-01    4.10
Length: 822, dtype: float64

In [11]:
#get Fama French 5 factor data from French's website
def getFamaFrench5(enddate=None):
    #enddate in integer yyyymm format if given    
    
    ffurl='http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_5_Factors_2x3_CSV.zip'

    #Read just the first line of the FF file into a dataframe
    df_monthly = pd.read_csv(ffurl, header=None, nrows=1)

    #Put the line into a string
    str=df_monthly.iloc[0,0]

    #8th word in the string is the last date in the file in YYYYMM format
    lastyear=int(str.split()[8][:4])
    lastmonth=int(str.split()[8][4:])
    #first date in the file is June 1926 - figure out
    #number of months based on that
    periods=(lastyear-1926)*12+(lastmonth-6)    
    
    #Use specified end date?
    if enddate:
        ed_year = int(enddate/100)
        ed_month = enddate%100
        ed_periods = (ed_year-1926)*12+(ed_month-6)
        if (ed_periods > 0) & (ed_periods < periods):    #Use specified end date
            periods = ed_periods
    
    #Now we know how many periods to read - skip the header and read those monthly periods
    names_monthly = ["yearmon", "mkt_minus_rf", "SMB", "HML", "RF", "RMW", "CMA"]
    df_monthly = pd.read_csv(ffurl, skiprows=4, nrows=periods, names=names_monthly)
    
    #Transfer from data frame to output arrays
    Date=df_monthly["yearmon"]
    market_minus_rf=df_monthly["mkt_minus_rf"]
    SMB=df_monthly["SMB"]
    HML=df_monthly["HML"]
    RF=df_monthly["RF"]
    RMW=df_monthly["RMW"]
    CMA=df_monthly["CMA"]
    
    return(Date,market_minus_rf,SMB,HML,RF, RMW, CMA)